In [2]:
import os



In [3]:
%pwd

'c:\\Users\\haari\\OneDrive\\Desktop\\aiml 2026\\Kidney-Disease-Classification-Deep-Learning-Project\\research\\research\\templates'

In [4]:
os.chdir("../../..")

In [5]:
%pwd

'c:\\Users\\haari\\OneDrive\\Desktop\\aiml 2026\\Kidney-Disease-Classification-Deep-Learning-Project'

In [6]:
import sys
from pathlib import Path

# get project root (folder that contains 'src')
PROJECT_ROOT = Path(r"C:\Users\haari\OneDrive\Desktop\aiml 2026\Kidney-Disease-Classification-Deep-Learning-Project")

# add src to python path
SRC_PATH = PROJECT_ROOT / "src"
if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))

print("PYTHONPATH set correctly ✅")
print(sys.path[0])


PYTHONPATH set correctly ✅
C:\Users\haari\OneDrive\Desktop\aiml 2026\Kidney-Disease-Classification-Deep-Learning-Project\src


In [7]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: int

In [8]:
import sys
from pathlib import Path

# get project root (folder that contains 'src')
PROJECT_ROOT = Path(r"C:\Users\haari\OneDrive\Desktop\aiml 2026\Kidney-Disease-Classification-Deep-Learning-Project")

# add src to python path
SRC_PATH = PROJECT_ROOT / "src"
if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))

print("PYTHONPATH set correctly ✅")
print(sys.path[0])


PYTHONPATH set correctly ✅
C:\Users\haari\OneDrive\Desktop\aiml 2026\Kidney-Disease-Classification-Deep-Learning-Project\src


In [9]:
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier.constants import *
import tensorflow as tf


In [10]:
import sys
from pathlib import Path

# get project root (folder that contains 'src')
PROJECT_ROOT = Path(r"C:\Users\haari\OneDrive\Desktop\aiml 2026\Kidney-Disease-Classification-Deep-Learning-Project")

# add src to python path
SRC_PATH = PROJECT_ROOT / "src"
if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))

print("PYTHONPATH set correctly ✅")
print(sys.path[0])


PYTHONPATH set correctly ✅
C:\Users\haari\OneDrive\Desktop\aiml 2026\Kidney-Disease-Classification-Deep-Learning-Project\src


In [11]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath: Path = CONFIG_FILE_PATH,
                 params_filepath: Path = PARAMS_FILE_PATH):
        
        # 1. Force the project root to your absolute path so it finds files from anywhere
        self.project_root = Path(
            r"C:\Users\haari\OneDrive\Desktop\aiml 2026\Kidney-Disease-Classification-Deep-Learning-Project"
        )

        # 2. Update the paths to use the project root
        # Check if your folder is named 'config' or 'configs' 
        self.config_filepath = self.project_root / "configs" / "config.yaml" 
        self.params_filepath = self.project_root / "params.yaml"

        # 3. Read the files using these absolute paths
        self.config = read_yaml(self.config_filepath)
        self.params = read_yaml(self.params_filepath)

        # 4. Create the artifacts root
        create_directories([Path(self.config.artifacts_root)])

    def get_training_config(self) -> TrainingConfig:
        training_config = self.config.training
        params = self.params
        
        # DIRECT PATHS - Hardcoded to your specific Windows folders
        # This bypasses all logic and relative pathing issues
        updated_base_model_path = r"C:\Users\haari\OneDrive\Desktop\aiml 2026\Kidney-Disease-Classification-Deep-Learning-Project\artifacts\prepare_base_model\base_model_updated.h5"
        
        training_data = r"C:\Users\haari\OneDrive\Desktop\aiml 2026\Kidney-Disease-Classification-Deep-Learning-Project\artifacts\data_ingestion\unzipped\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone"
        
        create_directories([Path(training_config.root_dir)])

        return TrainingConfig(
            root_dir=Path(training_config.root_dir),
            trained_model_path=Path(training_config.trained_model_path),
            updated_base_model_path=Path(updated_base_model_path), 
            training_data=Path(training_data),
            params_epochs=params.epochs,
            params_batch_size=params.batch_size,
            params_is_augmentation=params.is_augmentation, 
            params_image_size=list(params.image_size)
        )

In [12]:
import os
import tensorflow as tf
import urllib.request as request
from zipfile import ZipFile
import time
from pathlib import Path

In [13]:

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        
    def train_valid_generator(self):
        # FIX: Create a clean tuple (e.g., (224, 224)) for Keras
        img_size = tuple(map(int, self.config.params_image_size[:2]))

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=str(self.config.training_data),
            target_size=img_size, # FIXED
            batch_size=self.config.params_batch_size,
            class_mode='categorical',
            shuffle=False,
            interpolation="bilinear"
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                rotation_range=40,
                zoom_range=0.2,
                shear_range=0.2
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=str(self.config.training_data),
            target_size=img_size, # FIXED: Was causing the TypeError
            batch_size=self.config.params_batch_size,
            class_mode='categorical',
            shuffle=True,
            interpolation="bilinear"
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = max(
            1,
            self.train_generator.samples // self.train_generator.batch_size
        )
        self.validation_steps = max(
            1,
            self.valid_generator.samples // self.valid_generator.batch_size
        )

        print("Steps per epoch:", self.steps_per_epoch)
        print("Validation steps:", self.validation_steps)

        

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            validation_data=self.valid_generator,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            verbose=1 
        ) 

        if self.model is not None:
            self.save_model(
                path=self.config.trained_model_path,
                model=self.model
            )

In [14]:
import sys
from pathlib import Path

# get project root (folder that contains 'src')
PROJECT_ROOT = Path(r"C:\Users\haari\OneDrive\Desktop\aiml 2026\Kidney-Disease-Classification-Deep-Learning-Project")

# add src to python path
SRC_PATH = PROJECT_ROOT / "src"
if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))

print("PYTHONPATH set correctly ✅")
print(sys.path[0])


PYTHONPATH set correctly ✅
C:\Users\haari\OneDrive\Desktop\aiml 2026\Kidney-Disease-Classification-Deep-Learning-Project\src


In [15]:
config = ConfigurationManager()
print(dir(config))


2026-02-07 18:25:25,348 : INFO : cnnClassifier : yaml file: C:\Users\haari\OneDrive\Desktop\aiml 2026\Kidney-Disease-Classification-Deep-Learning-Project\configs\config.yaml loaded successfully


2026-02-07 18:25:25,362 : INFO : cnnClassifier : yaml file: C:\Users\haari\OneDrive\Desktop\aiml 2026\Kidney-Disease-Classification-Deep-Learning-Project\params.yaml loaded successfully
2026-02-07 18:25:25,366 : INFO : cnnClassifier : created directory at: artifacts
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'config', 'config_filepath', 'get_training_config', 'params', 'params_filepath', 'project_root']


In [16]:

config = ConfigurationManager()
training_config = config.get_training_config()
training = Training(config=training_config)
training.get_base_model()
training.train_valid_generator()
training.train()

2026-02-07 18:25:25,381 : INFO : cnnClassifier : yaml file: C:\Users\haari\OneDrive\Desktop\aiml 2026\Kidney-Disease-Classification-Deep-Learning-Project\configs\config.yaml loaded successfully
2026-02-07 18:25:25,386 : INFO : cnnClassifier : yaml file: C:\Users\haari\OneDrive\Desktop\aiml 2026\Kidney-Disease-Classification-Deep-Learning-Project\params.yaml loaded successfully
2026-02-07 18:25:25,388 : INFO : cnnClassifier : created directory at: artifacts
2026-02-07 18:25:25,389 : INFO : cnnClassifier : created directory at: artifacts\training
Found 12446 images belonging to 4 classes.
Found 12446 images belonging to 4 classes.
Steps per epoch: 777
Validation steps: 777
777/777 [==============================] - 17266s 22s/step - loss: 0.8296 - accuracy: 0.7008 - val_loss: 0.5256 - val_accuracy: 0.8061


c:\Users\haari\.conda\envs\kidney\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
import sys
print(sys.executable)

c:\Users\haari\.conda\envs\kidney\python.exe


In [18]:
%pip install Pillow
%pip install scipy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [28]:
import tensorflow as tf

model = tf.keras.models.load_model(
    r"artifacts/training/model.h5"
)

print("✅ Model loaded")


✅ Model loaded
